# DeepLabCut Toolbox - Demo BUCTD

Some resources that can be useful:

- [github.com/DeepLabCut/DeepLabCut](https://github.com/DeepLabCut/DeepLabCut)
- [DeepLabCut's Documentation: User Guide for Multi-Animal projects](https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html)
- [Zhou, Stoffl, Mathis, Mathis. "Rethinking Pose Estimation in Crowds: Overcoming the Detection Information Bottleneck and Ambiguity." Proceedings of the IEEE/CVF International Conference on Computer Vision (ICCV). 2023](https://openaccess.thecvf.com/content/ICCV2023/papers/Zhou_Rethinking_Pose_Estimation_in_Crowds_Overcoming_the_Detection_Information_Bottleneck_ICCV_2023_paper.pdf)

## Introduction

This notebook is an introduction to training and using CTD models in DeepLabCut, through the [maDLC Tri-Mouse Benchmark Dataset](https://zenodo.org/records/5851157) presented Lauer et al. 2022 (Nature Methods). For more information, you can check out the [DeepLabCut Benchmark Datasets](https://benchmark.deeplabcut.org/datasets.html) for more information.

In this notebook, we'll

- train an bottom-up model that can provide conditions for the CTD model
- evaluate the bottom-up model
- (optional/advanced) learn how the CTD model is trained with generative sampling
- train the CTD model
- evaluate the CTD model
- **Nice feature of CTD** use the CTD model to track individuals

Note: This notebook **can also be run locally**. However, using a GPU is recommended to train the models and run video inference. Just skip the _Installing DeepLabCut on COLAB_ section

### ⚠️⚠️ Change the Runtime type to use a GPU!⚠️⚠️

First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU".

### Installing DeepLabCut on COLAB

Let's install the latest version of DeepLabCut, straight from GitHub.

In [ ]:
!pip install "git+https://github.com/DeepLabCut/DeepLabCut.git#egg=deeplabcut"

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)** You will see this button at the output of the cells above ^.

### Imports

In [ ]:
import requests
import shutil
from io import BytesIO
from pathlib import Path
from zipfile import ZipFile

import deeplabcut
import deeplabcut.pose_estimation_pytorch as dlc_torch
import deeplabcut.utils.auxiliaryfunctions as auxiliaryfunctions
import matplotlib.pyplot as plt
import numpy as np

### Downloading the Tri-Mouse Dataset

This cell downloads the Tri-Mouse dataset from Zenodo into the current working directory (or `cwd`), which should be the directory you launched the jupyter server from.

In [ ]:
download_path = Path.cwd()
config = str(download_path / "trimice-dlc-2021-06-22" / "config.yaml")

In [ ]:
print(f"Downloading the tri-mouse dataset into {download_path}")

url_record = "https://zenodo.org/api/records/5851157"
response = requests.get(url_record)
if response.status_code == 200:
    file = response.json()["files"][0]
    title = file["key"]
    print(f"Downloading {title}...")
    with requests.get(file['links']['self'], stream=True) as r:
        with ZipFile(BytesIO(r.content)) as zf:
            zf.extractall(path=download_path)
else:
    raise ValueError(f"The URL {url_record} could not be reached.")


# Check that the config was downloaded correctly
print(f"Config path: {config}")
if not Path(config).exists():
    print(f"Could not find config at {config}: check that the dataset was downloaded correctly!")


## Training a CTD Model in DeepLabCut

BUCTD (or bottom-up conditioned top-down), as its name suggests, requires a bottom-up model to provide conditions (or **pose proposals**) for the CTD model to fix. So the first step in getting a CTD model that can be used to run inference is to train a bottom-up model to provide conditions!

We'll also **ensure that we're training the bottom-up and CTD models on the same train/test splits!** This is important: if you're training the models on different training images and evaluating them on different test images, then their results aren't comparable!

In [ ]:
cfg = auxiliaryfunctions.read_config(config)
train_frac = cfg["TrainingFraction"][0]
print(f"Using {int(100 * train_frac)}% of the data in the training set.")

num_images = 112
train_images = int(train_frac * num_images)

seed = 0
rng = np.random.default_rng(seed)

train_indices = rng.choice(num_images, size=train_images, replace=False, shuffle=False).tolist()
test_indices = [idx for idx in range(num_images) if idx not in train_indices]


### Training a BU Model

We'll take the simplest approach possible here and train a ResNet pose estimation model. As the CTD model will be used to improve the predictions made by the BU model, we want something light and fast rather than something heavy and slow!

We'll start by **creating the shuffle for the bottom-up model (with index 1) with the selected train/test split**.

In [ ]:
BU_SHUFFLE = 1

deeplabcut.create_training_dataset(
    config,
    Shuffles=[BU_SHUFFLE],
    trainIndices=[train_indices],
    testIndices=[test_indices],
    net_type="resnet_50",
    engine=deeplabcut.Engine.PYTORCH,
    userfeedback=False,
)

We can then train the model defined in the created bottom-up shuffle. To make running this notebook a bit quicker, we'll **only train the BU model for 100 epochs**. The model should still perform well enough, and as we're less interested in the BU model than the CTD model we'll save a bit of time and compute here. Training the model should **take 10 to 20 minutes**, depending on your CPU and GPU performance.


In [ ]:
deeplabcut.train_network(
    config,
    shuffle=BU_SHUFFLE,
    epochs=100,
)

And finally we evaluate it! If you trained for 100 epochs, you should get an mAP around 90, and RMSE around 4-5 pixels. When calling `evaluate_network`, the PAF graph is pruned (as described in [Lauer et al. 2022 (Nature Methods)](https://www.nature.com/articles/s41592-022-01443-0)) to boost performance.

In [ ]:
deeplabcut.evaluate_network(config, Shuffles=[BU_SHUFFLE])

### Training the CTD Model

As for the BU model, we need to start by creating the shuffle for the CTD model. We'll use `create_training_dataset_from_existing_split` to create a shuffle with the same train/test split as the BU shuffle. You could equivalently call `create_training_dataset(..., trainIndices=[train_indices], testIndices=[test_indices], ...)` again, as done above for the BU shuffle.

In this notebook, we'll use a preNet CTD architecture. You can check out the paper for more information on how preNet models are designed!

We'll also specify which model we want to use to provide conditions with the `ctd_conditions` parameter. As is indicated in the docstring:

```
ctd_conditions: int | str | Path | tuple[int, str] | tuple[int, int] | None, default = None,
  If using a conditional-top-down (CTD) net_type, this argument should be specified. It defines the
  conditions that will be used with the CTD model. It can be either:
    * A shuffle number (ctd_conditions: int), which must correspond to a bottom-up (BU) network type.
    * A predictions file path (ctd_conditions: string | Path), which must correspond to a 
      .json or .h5 predictions file.
    * A shuffle number and a particular snapshot (ctd_conditions: tuple[int, str] | tuple[int, int]), 
      which respectively correspond to a bottom-up (BU) network type and a particular snapshot name or 
      index.
```

We'll use the index of the BU shuffle defined above, and the best snapshot that was saved (indicated through a -1). You can edit which model is used to provide conditions through the `pytorch_config` for the `CTD_SHUFFLE` (in this case shuffle `2`):

```yaml
# Example: Loading the predictions for snapshot-250.pt of shuffle 1.
data:
  conditions:
    shuffle: 1
    snapshot: snapshot-250.pt

# Example: Loading the predictions for the last snapshot of shuffle 1.
data:
  conditions:
    shuffle: 1
    snapshot_index: -1
```

In [ ]:
CTD_SHUFFLE = 2

deeplabcut.create_training_dataset_from_existing_split(
    config,
    from_shuffle=BU_SHUFFLE,
    shuffles=[CTD_SHUFFLE],
    net_type="ctd_prenet_cspnext_m",
    engine=deeplabcut.Engine.PYTORCH,
    ctd_conditions=(BU_SHUFFLE, -1),
)

#### (Optional/Advanced) Learning and visualizing generative sampling during training

You can skip this section as it's simply to visualize how CTD models are trained, if you aren't interested in learning about it. This section **uses some internal DeepLabCut functions which may seem a bit complicated if you're not used to using them; you can ignore most of the code and just read the text/comments and look at the outputs if you're more comfortable with that.**

Conditional top-down models are trained using _generative sampling_, as introduced in PoseFix \[1\]. For every ground truth pose, we'll add some errors. The errors that can be introduced are:

- Jitter error is defined as a small displacement from the GT keypoint.
- Swap error represents a confusion between the same or similar parts which belong to different persons.
- Inversion error occurs when a pose estimation model is confused between semantically similar parts that belong to the same instance.
- Miss error represents a large displacement from the GT keypoint position.

It's important that "enough" generative sampling is applied (so the model can learn how to correct errors), but applying too much can be bad too! You want the model to learn to correct errors that are realistic (w.r.t. the task at hand), not just receive random points and have to learn by itself where the keypoints go. **The default parameters should work well on most datasets.**

The way these keypoints are "sampled" can be visuallized below. We'll create a `dataset` (which is used by DeepLabCut for training) and sample some data from this dataset. You can see that every time we sample an image, we get different keypoint conditions that will be given to the model. This ensures that the model is well trained to deal with a variety of mistakes that can be made by the bottom up model. On the left side of the plots, you have an image with the ground truth keypoints annotated. On the right side of the plots, you have the pose conditions that the CTD model will receive and will be tasked with fixing.

> \[1\]: Moon, Gyeongsik, Ju Yong Chang, and Kyoung Mu Lee. "Posefix: Model-agnostic general human pose refinement network." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2019

In [ ]:
def plot_generative_sampling(dataset: dlc_torch.PoseDataset) -> None:
    # Sample the same image 3 times and plot the results
    for i in range(3):
        item = dataset[0]

        # Remove ImageNet normalization from the image so it displays well
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = item["image"].transpose((1, 2, 0))
        img = np.clip(img * std + mean, 0, 1)

        # Get the ground trouth and "conditional pose"
        gt_pose = item["annotations"]["keypoints"][0]
        gen_samples = item["context"]["cond_keypoints"][0]

        fig, axs = plt.subplots(1, 2, figsize=(8, 4))
        for ax in axs:
            ax.imshow(img)
            ax.axis("off")

        # plot the ground truth on the left and conditions on the right
        for ax, title, keypoints in zip(
            axs,
            ["Ground Truth Pose", "Pose Conditions"],
            [gt_pose, gen_samples],
        ):
            ax.set_title(title)
            for x, y, vis in keypoints:
                if vis > 0:
                    ax.scatter([x], [y])


ctd_loader = dlc_torch.DLCLoader(config, shuffle=CTD_SHUFFLE)

transform = dlc_torch.build_transforms(ctd_loader.model_cfg["data"]["train"])
dataset = ctd_loader.create_dataset(transform, mode="train", task=ctd_loader.pose_task)

# Fix the seeds for reproducibility; you can change the seed from `0` to another value
# to change the results
dlc_torch.fix_seeds(0)
plot_generative_sampling(dataset)

The generative sampling can be parameterized through the `pytorch_config.yaml` as well. Let's play around with these parameters a bit and see how that changes the conditions that will be given to the model.

First, we'll just lower the `keypoint_sigmas`, which impacts how much pose conditions can move during jittering.

In [ ]:
ctd_loader = dlc_torch.DLCLoader(config, shuffle=CTD_SHUFFLE)

# We'll edit the model config here directly; In practice, edit the pytorch_config file instead.
# The parameters that can be set here are the parameters of the `dlc_torch.GenSamplingConfig`
ctd_loader.model_cfg["data"]["gen_sampling"] = {
    # lower the keypoint sigma by a factor of 2 (default: 0.1)
    #   -> this changes by how much keypoints are jittered; the smaller 
    #      the value, the smaller the jitter
    "keypoint_sigmas": 0.05,
}

transform = dlc_torch.build_transforms(ctd_loader.model_cfg["data"]["train"])
dataset = ctd_loader.create_dataset(transform, mode="train", task=ctd_loader.pose_task)

# Fix the seeds for reproducibility; you can change the seed from `0` to another value
# to change the results
dlc_torch.fix_seeds(0)
plot_generative_sampling(dataset)

Next, we'll update the probabilities of make errors. You can edit these values yourself to see how it impacts the generative sampling. Note that these probabilities are **not absolute** - as a single type of error is applied to each keypoint, changing the probability of one type of error happening will change the probability that other types of errors occur.

In [ ]:
ctd_loader = dlc_torch.DLCLoader(config, shuffle=CTD_SHUFFLE)

# We'll edit the model config here directly; In practice, edit the pytorch_config file instead.
# The parameters that can be set here are the parameters of the `dlc_torch.GenSamplingConfig`
ctd_loader.model_cfg["data"]["gen_sampling"] = {
    "jitter_prob": 0.5,
    "swap_prob": 0.1,
    "inv_prob": 0.1,
    "miss_prob": 0.25,
}

transform = dlc_torch.build_transforms(ctd_loader.model_cfg["data"]["train"])
dataset = ctd_loader.create_dataset(transform, mode="train", task=ctd_loader.pose_task)

# Fix the seeds for reproducibility; you can change the seed from `0` to another value
# to change the results
dlc_torch.fix_seeds(0)
plot_generative_sampling(dataset)

#### Training and Evaluating the CTD Model

Next, we can simply train the CTD model. It should take **15 to 30 minutes** to train the model to 250 epochs on a GPU.

In [ ]:
deeplabcut.train_network(config, shuffle=CTD_SHUFFLE)

If your CTD model is well trained, it should now outperform the performance of the BU model who's predictions it uses as conditions!

In [ ]:
deeplabcut.evaluate_network(config, Shuffles=[CTD_SHUFFLE])

## Tracking with CTD

One of the big advantages of having a CTD model is that it can be used to track individuals directly! Let's say you have the pose for your animals at `frame T`. Then you can use those poses as conditions for `frame T+1`, and let your CTD model simply "update" the poses depending on how much your mice moved.

In the simplest scenario, you only need to run the BU model on the first frame, and then the CTD model takes over for inference and tracking:

1. Run the BU model to generate conditions for the 1st frame of the video
2. For every frame after that, use the predictions from the previous frame as conditions

However, this may not fit your scenario perfectly. Maybe all the mice aren't present in the first frame, and if they aren't detected by the BU model they'll never be tracked. Maybe at some point the CTD model makes an error and you lose track of a mouse. There are some options to deal with this:

- Run the BU model every time at least one mouse is not detected (if you expect N mice to be in the video and you only detect N-1 mice, run the BU model):
  - In this case, the predictions from the BU model need to be "merged in" to the existing N-1 tracks
  - We can merge them in by using a similarity score between poses (OKS) which ranges from 0 to 1
  - You likely don't want to run the BU model every frame, as this would slow down inference.
- Run the BU model every K frames in case new mice appear


### Downloading a Tri-Mouse video

First, let's download a video from the 3-mice dataset.

In [ ]:
download_path = Path.cwd()
video_name = "videocompressed1.mp4"
video_path = str(download_path / video_name)
print(f"Video will be saved in {video_path}")

In [ ]:
print(f"Downloading the tri-mouse video into {download_path}")

url_video_record = "https://zenodo.org/api/records/7883589"
response = requests.get(url_video_record)
if response.status_code == 200:
    file = response.json()["files"][0]
    title = file["key"]
    print(f"Downloading {title}...")
    with requests.get(file['links']['self'], stream=True) as r:
        with ZipFile(BytesIO(r.content)) as zf:
            zf.extractall(path=download_path)
else:
    raise ValueError(f"The URL {url_record} could not be reached.")

# Check that the video was downloaded
src_video_path = download_path / "demo-me-2021-07-14" / "videos" / video_name
if not src_video_path.exists():
    raise ValueError("Failed to download the video")

# Move the video to the final path
shutil.move(src_video_path, video_path)
if not Path(video_path).exists():
    raise ValueError("Failed to move the video")


### Running Video Analysis

You can track using your CTD model by setting `ctd_tracking=True` when calling `analyze_videos`. Of course, you then won't need to convert detections to tracklets or link tracklets, as the CTD model will directly be tracking the animals.

You can visualize the outputs by setting `track_method="ctd"` when calling `create_labeled_video`.

In [ ]:
deeplabcut.analyze_videos(
    config,
    [video_path],
    shuffle=CTD_SHUFFLE,
    ctd_tracking=True,
)
deeplabcut.create_labeled_video(
    config,
    [video_path],
    shuffle=CTD_SHUFFLE,
    track_method="ctd",
    color_by="individual",
)

It can be beneficial to customize the tracking parameters a bit. The tracking paramaters you can set are:

- **`bu_on_lost_idv`**: When True, the BU model is run when there are fewer conditions found than the expected number of individuals in the video.
- **`bu_min_frequency`**: The minimum frequency at which the BU model is run to generate conditions. If None, the BU model is only run to initialize the pose in the first frame, and then is not run again. If a positive number N, the BU model is run every N frames. The BU predictions are then combined with the CTD predictions to continue the tracklets.
- **`bu_max_frequency`**: The maximum frequency at which the BU model can be run. Must be greater than `bu_min_frequency`. When there are fewer conditions than individuals expected in the video and `bu_on_lost_idv` is True, the BU model may be run on every frame. This can happen if individuals can disappear from the video, and each frame may have a variable number of individuals. If `bu_max_frequency` is set to N, then the BU model will be run at most every N-th frame, which improves the inference speed of the model.
- **`threshold_bu_add`**: The OKS threshold below which a BU pose must be (wrt. any existing CTD pose) to be added to the poses.
- **`threshold_ctd`**: The score threshold below which detected keypoints are NOT given to the CTD model to predict pose for the next frame.
- **`threshold_nms`**: The OKS threshold for non-maximum suppression to remove duplicates poses when two CTD model predictions converge to a single animal.


In [ ]:
dest_folder = str(Path(video).parent / "custom-ctd-tracking")

deeplabcut.analyze_videos(
    config,
    [video_path],
    shuffle=CTD_SHUFFLE,
    destfolder=dest_folder,
    ctd_tracking=dict(
        bu_on_lost_idv=True,
        bu_max_frequency=10,
        threshold_bu_add=0.5,
        threshold_ctd=0.01,
        threshold_nms=0.8,
    ),
)
deeplabcut.create_labeled_video(
    config,
    [video_path],
    shuffle=CTD_SHUFFLE,
    destfolder=dest_folder,
    track_method="ctd",
    color_by="individual",
)